In [1]:
import json

In [2]:
with open("unsrtuctured_result_data.json", "r") as json_file:
    data = json.load(json_file)

In [3]:
Header = data.get("Header", [])
Footer = data.get("Footer", [])
Title = data.get("Title", [])
NarrativeText = data.get("NarrativeText", [])
Text = data.get("Text", [])
ListItem = data.get("ListItem", [])
img = data.get("img", [])
tab = data.get("tab", [])

In [4]:
len(NarrativeText)

2358

In [5]:
type(data)

dict

In [6]:
chunked_text = []

In [7]:
token_length = [len(item) // 4 for item in NarrativeText]

In [8]:
# token_length

In [9]:
for i, length in enumerate(token_length, 1):
    print(f"Item {i-1}: {length} tokens")

Item 0: 8 tokens
Item 1: 317 tokens
Item 2: 242 tokens
Item 3: 135 tokens
Item 4: 344 tokens
Item 5: 113 tokens
Item 6: 57 tokens
Item 7: 272 tokens
Item 8: 163 tokens
Item 9: 457 tokens
Item 10: 136 tokens
Item 11: 111 tokens
Item 12: 102 tokens
Item 13: 187 tokens
Item 14: 227 tokens
Item 15: 39 tokens
Item 16: 178 tokens
Item 17: 69 tokens
Item 18: 217 tokens
Item 19: 64 tokens
Item 20: 148 tokens
Item 21: 132 tokens
Item 22: 314 tokens
Item 23: 116 tokens
Item 24: 61 tokens
Item 25: 317 tokens
Item 26: 96 tokens
Item 27: 57 tokens
Item 28: 309 tokens
Item 29: 297 tokens
Item 30: 35 tokens
Item 31: 332 tokens
Item 32: 108 tokens
Item 33: 92 tokens
Item 34: 32 tokens
Item 35: 154 tokens
Item 36: 63 tokens
Item 37: 299 tokens
Item 38: 127 tokens
Item 39: 178 tokens
Item 40: 244 tokens
Item 41: 136 tokens
Item 42: 226 tokens
Item 43: 181 tokens
Item 44: 38 tokens
Item 45: 288 tokens
Item 46: 276 tokens
Item 47: 397 tokens
Item 48: 76 tokens
Item 49: 159 tokens
Item 50: 246 tokens
Item 

In [10]:
total_tokens = sum(len(item) // 4 for item in NarrativeText)

In [11]:
average_token_length = total_tokens / len(NarrativeText) if NarrativeText else 0

In [12]:
average_token_length

103.85072094995759

In [13]:
max(token_length)

1074

In [14]:
NarrativeText[165]

'ss Poy'

In [15]:
# Download PDF file
import os
import requests

# Get PDF document
pdf_path = "data-pdf/Gastrointestinal and Liver Disease.pdf"

# Download PDF if it doesn't already exist
if not os.path.exists(pdf_path):
  print("File doesn't exist, downloading...")

  # The URL of the PDF you want to download
  url = "https://pressbooks.oer.hawaii.edu/humannutrition2/open/download?type=pdf"

  # The local filename to save the downloaded file
  filename = pdf_path

  # Send a GET request to the URL
  response = requests.get(url)

  # Check if the request was successful
  if response.status_code == 200:
      # Open a file in binary write mode and save the content to it
      with open(filename, "wb") as file:
          file.write(response.content)
      print(f"The file has been downloaded and saved as {filename}")
  else:
      print(f"Failed to download the file. Status code: {response.status_code}")
else:
  print(f"File {pdf_path} exists.")

File data-pdf/Gastrointestinal and Liver Disease.pdf exists.


In [16]:
import fitz # (pymupdf, found this is better than pypdf for our use case, note: licence is AGPL-3.0, keep that in mind if you want to use any code commercially)
from tqdm.auto import tqdm # for progress bars, requires !pip install tqdm 

def text_formatter(text: str) -> str:
    """Performs minor formatting on text."""
    cleaned_text = text.replace("\n", " ").strip() # note: this might be different for each doc (best to experiment)

    # Other potential text formatting functions can go here
    return cleaned_text

# Open PDF and get lines/pages
# Note: this only focuses on text, rather than images/figures etc
def open_and_read_pdf(pdf_path: str) -> list[dict]:
    """
    Opens a PDF file, reads its text content page by page, and collects statistics.

    Parameters:
        pdf_path (str): The file path to the PDF document to be opened and read.

    Returns:
        list[dict]: A list of dictionaries, each containing the page number
        (adjusted), character count, word count, sentence count, token count, and the extracted text
        for each page.
    """
    doc = fitz.open(pdf_path)  # open a document
    pages_and_texts = []
    for page_number, page in tqdm(enumerate(doc)):  # iterate the document pages
        text = page.get_text()  # get plain text encoded as UTF-8
        text = text_formatter(text)
        pages_and_texts.append({"page_number": page_number,
                                "page_char_count": len(text),
                                "page_word_count": len(text.split(" ")),
                                "page_sentence_count_raw": len(text.split(". ")),
                                "page_token_count": len(text) / 4,  # 1 token = ~4 chars, see: https://help.openai.com/en/articles/4936856-what-are-tokens-and-how-to-count-them
                                "text": text})
    return pages_and_texts

pages_and_texts = open_and_read_pdf(pdf_path=pdf_path)
pages_and_texts[:2]

0it [00:00, ?it/s]

[{'page_number': 0,
  'page_char_count': 6877,
  'page_word_count': 941,
  'page_sentence_count_raw': 36,
  'page_token_count': 1719.25,
  'text': '806 Peptic Ulcer Disease Francis K.L. Chan, James Y.W. Lau 53 An ulcer in the GI tract can be defined as a 5 mm or larger break in  the lining of the mucosa, with appreciable depth at endoscopy or  with histologic evidence of submucosal extension. An erosion is a  break less than 5 mm. The distinction between an ulcer and an ero\xad sion is somewhat arbitrary. The term PUD is used to include ulcer\xad ations and erosions in the stomach and duodenum from a number  of causes. These lesions are called “peptic” because the enzyme  pepsin, proteolytic at an acidic pH (see Chapter 51), plays a major  role in causing the mucosal breaks, regardless of the inciting agent. Decades of research focused on the role of gastric acid  secretion and the effects of stress, personality type, and  genetics in the pathogenesis of PUD. The discovery of the  hist

In [17]:
import pandas as pd

df = pd.DataFrame(pages_and_texts)
df.head()

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,text
0,0,6877,941,36,1719.25,"806 Peptic Ulcer Disease Francis K.L. Chan, Ja..."
1,1,7559,1241,41,1889.75,CHAPTER 53 Peptic Ulcer Disease 807 53 Hp Inf...
2,2,5241,868,39,1310.25,PART VI Stomach and Duodenum 808 Hp infection...
3,3,3482,552,24,870.50,CHAPTER 53 Peptic Ulcer Disease 809 53 Dyspep...
4,4,7831,1242,50,1957.75,PART VI Stomach and Duodenum 810 According to...


In [18]:
# Get stats
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count
count,272.00,272.00,272.00,272.00,272.00
mean,135.50,5209.60,665.68,43.67,1302.40
std,78.66,2164.55,409.64,52.51,541.14
min,0.00,244.00,16.00,3.00,61.00
25%,67.75,3757.50,280.75,13.75,939.38
50%,135.50,5255.00,729.00,28.00,1313.75
75%,203.25,6373.25,961.25,46.00,1593.31
max,271.00,12933.00,1798.00,355.00,3233.25


In [20]:
from spacy.lang.en import English # see https://spacy.io/usage for install instructions

nlp = English()

# Add a sentencizer pipeline, see https://spacy.io/api/sentencizer/ 
nlp.add_pipe("sentencizer")

In [21]:
for item in tqdm(pages_and_texts):
    item["sentences"] = list(nlp(item["text"]).sents)
    
    # Make sure all sentences are strings
    item["sentences"] = [str(sentence) for sentence in item["sentences"]]
    
    # Count the sentences 
    item["page_sentence_count_spacy"] = len(item["sentences"])

  0%|          | 0/272 [00:00<?, ?it/s]

In [23]:
import random
# Inspect an example
random.sample(pages_and_texts, k=1)

[{'page_number': 161,
  'page_char_count': 1545,
  'page_word_count': 220,
  'page_sentence_count_raw': 11,
  'page_token_count': 386.25,
  'text': 'authors received support (commercial/academic cooperation): Aji- nomoto Pharmaceuticals Co., Ltd., AsTellas Pharma Inc., AstraZe- necaK. K., Bayer Yakuhin, Ltd., Chugai PharmaCeutical Co., Ltd., DaiichiSankyo Company, Limited, Eisai Co., Ltd., Kishuhosokawa Co., Ltd., Maruso Co., Ltd, Mitsubishi Tanabe Pharma Corporation, MSD K.K., Nihon Pharmaceutical Co. Ltd., Nippon Shinyaku Co., Ltd., Okahatanoen Co., Ltd., Otsuka Pharmaceutical Co., Ltd., Pﬁzer Japan Inc., SanoﬁK.K., Taiho Pharmaceutical Co., Ltd., Takeda Pharmaceutical. Open Access This article is licensed under a Creative Commons Attribution 4.0 International License, which permits use, sharing, adaptation, distribution and reproduction in any medium or format, as long as you give appropriate credit to the original author(s) and the source, provide a link to the Creative Commons lic

In [24]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy
count,272.00,272.00,272.00,272.00,272.00,272.00
mean,135.50,5209.60,665.68,43.67,1302.40,48.88
std,78.66,2164.55,409.64,52.51,541.14,61.97
min,0.00,244.00,16.00,3.00,61.00,4.00
25%,67.75,3757.50,280.75,13.75,939.38,20.00
50%,135.50,5255.00,729.00,28.00,1313.75,32.50
75%,203.25,6373.25,961.25,46.00,1593.31,46.25
max,271.00,12933.00,1798.00,355.00,3233.25,636.00


In [28]:
# Define split size to turn groups of sentences into chunks
num_sentence_chunk_size = 43

# Create a function that recursively splits a list into desired sizes
def split_list(input_list: list, 
               slice_size: int) -> list[list[str]]:
    """
    Splits the input_list into sublists of size slice_size (or as close as possible).

    For example, a list of 17 sentences would be split into two lists of [[10], [7]]
    """
    return [input_list[i:i + slice_size] for i in range(0, len(input_list), slice_size)]

# Loop through pages and texts and split sentences into chunks
for item in tqdm(pages_and_texts):
    item["sentence_chunks"] = split_list(input_list=item["sentences"],
                                         slice_size=num_sentence_chunk_size)
    item["num_chunks"] = len(item["sentence_chunks"])

  0%|          | 0/272 [00:00<?, ?it/s]

In [ ]:
random.sample(pages_and_texts, k=1)

In [30]:
df = pd.DataFrame(pages_and_texts)
df.describe().round(2)

,page_number,page_char_count,page_word_count,page_sentence_count_raw,page_token_count,page_sentence_count_spacy,num_chunks
count,272.00,272.00,272.00,272.00,272.00,272.00,272.00
mean,135.50,5209.60,665.68,43.67,1302.40,48.88,1.62
std,78.66,2164.55,409.64,52.51,541.14,61.97,1.44
min,0.00,244.00,16.00,3.00,61.00,4.00,1.00
25%,67.75,3757.50,280.75,13.75,939.38,20.00,1.00
50%,135.50,5255.00,729.00,28.00,1313.75,32.50,1.00
75%,203.25,6373.25,961.25,46.00,1593.31,46.25,2.00
max,271.00,12933.00,1798.00,355.00,3233.25,636.00,15.00


In [31]:
import re

# Split each chunk into its own item
pages_and_chunks = []
for item in tqdm(pages_and_texts):
    for sentence_chunk in item["sentence_chunks"]:
        chunk_dict = {}
        chunk_dict["page_number"] = item["page_number"]
        
        # Join the sentences together into a paragraph-like structure, aka a chunk (so they are a single string)
        joined_sentence_chunk = "".join(sentence_chunk).replace("  ", " ").strip()
        joined_sentence_chunk = re.sub(r'\.([A-Z])', r'. \1', joined_sentence_chunk) # ".A" -> ". A" for any full-stop/capital letter combo 
        chunk_dict["sentence_chunk"] = joined_sentence_chunk

        # Get stats about the chunk
        chunk_dict["chunk_char_count"] = len(joined_sentence_chunk)
        chunk_dict["chunk_word_count"] = len([word for word in joined_sentence_chunk.split(" ")])
        chunk_dict["chunk_token_count"] = len(joined_sentence_chunk) / 4 # 1 token = ~4 characters
        
        pages_and_chunks.append(chunk_dict)

# How many chunks do we have?
len(pages_and_chunks)

  0%|          | 0/272 [00:00<?, ?it/s]

440

In [ ]:
random.sample(pages_and_chunks, k=1)

In [33]:
# Get stats about our chunks
df = pd.DataFrame(pages_and_chunks)
df.describe().round(2)

,page_number,chunk_char_count,chunk_word_count,chunk_token_count
count,440.00,440.00,440.00,440.00
mean,130.87,3192.50,383.99,798.13
std,81.21,2064.17,323.17,516.04
min,0.00,6.00,1.00,1.50
25%,63.00,1601.00,119.75,400.25
50%,125.00,2609.00,273.50,652.25
75%,201.25,5040.25,665.75,1260.06
max,271.00,8379.00,1268.00,2094.75


In [37]:
# Show random chunks with under 30 tokens in length
min_token_length = 30
for row in df[df["chunk_token_count"] <= min_token_length].sample(10).iterrows():
    print(f'Chunk token count: {row[1]["chunk_token_count"]} | Text: {row[1]["sentence_chunk"]}')

Chunk token count: 8.5 | Text: Elsevier Inc. All rights reserved.
Chunk token count: 29.5 | Text: 11 • 3 December 2019 Annals.org Downloaded from https://annals.org by Yonsei University Medical Library on 05/29/2024.
Chunk token count: 5.0 | Text: All rights reserved.
Chunk token count: 23.5 | Text: http://www.bmj.com/ BMJ: first published as 10.1136/bmj.l536 on 25 March 2019. Downloaded from
Chunk token count: 26.25 | Text: �.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�815 Perforation.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.
Chunk token count: 1.5 | Text: Proton
Chunk token count: 29.0 | Text: �.�.�.�.�.�813 Misoprosto.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�813 PPIs.�.�.�.�.�.
Chunk token count: 22.5 | Text: All Rights Reserved.   Terms of Use • Privacy Policy • Notice • Accessibility Page 37 / 49
Chunk token count: 17.75 | Text: Copyright © 2012. John Wiley & Sons, Incorporated. All rights reserved.
Chunk token count: 12.75 | Text: Copyright ©2024. Elsevi

In [38]:
pages_and_chunks_over_min_token_len = df[df["chunk_token_count"] > min_token_length].to_dict(orient="records")
pages_and_chunks_over_min_token_len[:2]

[{'page_number': 0,
  'sentence_chunk': '806 Peptic Ulcer Disease Francis K. L. Chan, James Y. W. Lau 53 An ulcer in the GI tract can be defined as a 5 mm or larger break in the lining of the mucosa, with appreciable depth at endoscopy or with histologic evidence of submucosal extension. An erosion is a break less than 5 mm. The distinction between an ulcer and an ero\xad sion is somewhat arbitrary. The term PUD is used to include ulcer\xad ations and erosions in the stomach and duodenum from a number of causes. These lesions are called “peptic” because the enzyme pepsin, proteolytic at an acidic pH (see Chapter 51), plays a major role in causing the mucosal breaks, regardless of the inciting agent. Decades of research focused on the role of gastric acid secretion and the effects of stress, personality type, and genetics in the pathogenesis of PUD. The discovery of the histamine-2 (H2) receptor and development of H2RAs,1 and subsequently PPIs, led to major changes in the management of 

In [54]:
for item in tqdm(pages_and_chunks_over_min_token_len):
    item["embedding"] = embedding_model.encode(item["sentence_chunk"])

  0%|          | 0/422 [00:00<?, ?it/s]

In [47]:
from sentence_transformers import SentenceTransformer

# This model supports two prompts: "s2p_query" and "s2s_query" for sentence-to-passage and sentence-to-sentence tasks, respectively.
# They are defined in `config_sentence_transformers.json`
query_prompt_name = "s2p_query"

# ！The default dimension is 1024, if you need other dimensions, please clone the model and modify `modules.json` to replace `2_Dense_1024` with another dimension, e.g. `2_Dense_256` or `2_Dense_8192` !
# on gpu
embedding_model = SentenceTransformer("dunzhang/stella_en_400M_v5", trust_remote_code=True).cuda()



You try to use a model that was created with version 3.0.1, however, your version is 2.7.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.



Some weights of the model checkpoint at dunzhang/stella_en_400M_v5 were not used when initializing NewModel: ['new.pooler.dense.bias', 'new.pooler.dense.weight']
- This IS expected if you are initializing NewModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing NewModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [55]:
# Turn text chunks into a single list
text_chunks = [item["sentence_chunk"] for item in pages_and_chunks_over_min_token_len]

In [56]:
%%time

# Embed all texts in batches
text_chunk_embeddings = embedding_model.encode(text_chunks,
                                               batch_size=32, # you can use different batch sizes here for speed/performance, I found 32 works well for this use case
                                               convert_to_tensor=True) # optional to return embeddings as tensor instead of array

text_chunk_embeddings

CPU times: user 17.9 s, sys: 66.2 ms, total: 17.9 s
Wall time: 14.4 s


tensor([[-0.4431, -0.1950, -1.0122,  ...,  0.2035, -0.0111, -1.4255],
        [ 0.1477, -0.1167, -2.2881,  ..., -0.4011,  0.0090, -0.9532],
        [ 0.1258, -0.6078, -2.7702,  ...,  0.1600,  0.9733, -1.6094],
        ...,
        [ 0.0405, -0.2877, -2.4515,  ...,  0.6552,  1.0580, -0.8779],
        [ 0.0481, -0.6341, -2.0686,  ...,  0.5455,  1.2294, -1.0640],
        [ 0.2839, -0.6972, -1.4129,  ...,  0.7671,  0.2448, -1.5266]],
       device='cuda:0')

In [57]:
from accelerate import Accelerator
from accelerate.utils import gather_object

accelerator = Accelerator()

# each GPU creates a string
message=[ f"Hello this is GPU {accelerator.process_index}" ] 

# collect the messages from all GPUs
messages=gather_object(message)

# output the messages only on the main process with accelerator.print() 
accelerator.print(messages)

['Hello this is GPU 0']


In [58]:
# Save embeddings to file
text_chunks_and_embeddings_df = pd.DataFrame(pages_and_chunks_over_min_token_len)
embeddings_df_save_path = "text_chunks_and_embeddings_df.csv"
text_chunks_and_embeddings_df.to_csv(embeddings_df_save_path, index=False)

In [59]:
# Import saved file and view
text_chunks_and_embedding_df_load = pd.read_csv(embeddings_df_save_path)
text_chunks_and_embedding_df_load.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,0,"806 Peptic Ulcer Disease Francis K. L. Chan, J...",4736,775,1184.00,[-0.443083 -0.194976 -1.0122081 ... 0.20...
1,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�806 ETIOLOG...,137,6,34.25,[ 0.14773934 -0.11674534 -2.2880774 ... -0.40...
2,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�....,124,7,31.00,[ 0.12582467 -0.60779524 -2.770172 ... 0.16...
3,0,�807 Other Causes of Ulcers and Idiopathic Ulc...,228,19,57.00,[ 0.19653367 -0.6044692 -1.5477078 ... 0.54...
4,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�810 Pharmaceu...,142,6,35.50,[ 0.1747831 -0.07459477 -2.1620371 ... 0.47...


In [60]:
import random

import torch
import numpy as np 
import pandas as pd

device = "cuda" if torch.cuda.is_available() else "cpu"

# Import texts and embedding df
text_chunks_and_embedding_df = pd.read_csv("text_chunks_and_embeddings_df.csv")

# Convert embedding column back to np.array (it got converted to string when it got saved to CSV)
text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))

# Convert texts and embedding df to list of dicts
pages_and_chunks = text_chunks_and_embedding_df.to_dict(orient="records")

# Convert embeddings to torch tensor and send to device (note: NumPy arrays are float64, torch tensors are float32 by default)
embeddings = torch.tensor(np.array(text_chunks_and_embedding_df["embedding"].tolist()), dtype=torch.float32).to(device)
embeddings.shape

/tmp/ipykernel_79267/4062955739.py:13: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  text_chunks_and_embedding_df["embedding"] = text_chunks_and_embedding_df["embedding"].apply(lambda x: np.fromstring(x.strip("[]"), sep=" "))


torch.Size([422, 3])

In [61]:
text_chunks_and_embedding_df.head()

,page_number,sentence_chunk,chunk_char_count,chunk_word_count,chunk_token_count,embedding
0,0,"806 Peptic Ulcer Disease Francis K. L. Chan, J...",4736,775,1184.00,"[-0.443083, -0.194976, -1.0122081]"
1,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�806 ETIOLOG...,137,6,34.25,"[0.14773934, -0.11674534, -2.2880774]"
2,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�....,124,7,31.00,"[0.12582467, -0.60779524, -2.770172]"
3,0,�807 Other Causes of Ulcers and Idiopathic Ulc...,228,19,57.00,"[0.19653367, -0.6044692, -1.5477078]"
4,0,�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�.�810 Pharmaceu...,142,6,35.50,"[0.1747831, -0.07459477, -2.1620371]"


In [67]:
import sentence_transformers
# 1. Define the query
# Note: This could be anything. But since we're working with a nutrition textbook, we'll stick with nutrition-based queries.
query = "How long does it typically take for a gastric ulcer to heal?"
print(f"Query: {query}")

# 2. Embed the query to the same numerical space as the text examples 
# Note: It's important to embed your query with the same model you embedded your examples with.
query_embedding = embedding_model.encode(query, convert_to_tensor=True)

# 3. Get similarity scores with the dot product (we'll time this for fun)
from time import perf_counter as timer

start_time = timer()
dot_scores = sentence_transformers.util.dot_score(a=query_embedding, b=embeddings)[0]
end_time = timer()

print(f"Time take to get scores on {len(embeddings)} embeddings: {end_time-start_time:.5f} seconds.")

# 4. Get the top-k results (we'll keep this to 5)
top_results_dot_product = torch.topk(dot_scores, k=5)
top_results_dot_product

Query: How long does it typically take for a gastric ulcer to heal?


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x1024 and 3x422)